## Optimizadores, Learning Rate Schedulers

En esta sesión vamos a comparar el resultado diferentes optimizadores y learning rates schedulers

## Leer, normalizar y particionar datos

In [2]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 784)
val set (12000, 784)


## Optimizadores

Vamos a probar diferentes optimizadores:

In [ ]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD,Adam,Adagrad

batch_size=128
epochs=5
lr=0.001

opt=[]
opt.append(SGD(learning_rate=lr, momentum=0.9))
opt.append(Adam(learning_rate=lr))
opt.append(Adagrad(learning_rate=lr))
#optimizador AdamW es muy bueno para transformers
best_acc=0.0
for optim in opt:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])



    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))

    print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")

    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        bestopt=optim

print("=============================")
print("Best acc",best_acc)
print("Best optim",bestopt)
print("=============================")


## **EJERCICIO**

Añade más epochs a este ejemplo anterior, un early stopping y un model_checkpoint. Luego prueba el test sobre el model checkpoint almacenado.

In [3]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD,Adam,Adagrad

batch_size=128
epochs=100
lr=0.01

opt=[]
opt.append(SGD(learning_rate=lr, momentum=0.9))
opt.append(Adam(learning_rate=lr))
opt.append(Adagrad(learning_rate=lr))
#optimizador AdamW es muy bueno para transformers
best_acc=0.0
i=0
for optim in opt:
    model = Sequential()
    i=i+1
    model.add(Input(784))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
    checkpoint = keras.callbacks.ModelCheckpoint(filepath="best_model"+str(i)+".h5", monitor='val_accuracy', save_best_only=True, verbose=1)

    model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])



    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback,checkpoint])

    print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")

    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        bestopt=optim

print("=============================")
print("Best acc",best_acc)
print("Best optim",bestopt)
print("=============================")

Epoch 1/100
362/375 [===========================>..] - ETA: 0s - loss: 0.4701 - accuracy: 0.8684
Epoch 1: val_accuracy improved from -inf to 0.93300, saving model to best_model1.h5
375/375 [==============================] - 4s 5ms/step - loss: 0.4632 - accuracy: 0.8702 - val_loss: 0.2369 - val_accuracy: 0.9330
Epoch 2/100
 21/375 [>.............................] - ETA: 1s - loss: 0.2317 - accuracy: 0.9379

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


368/375 [============================>.] - ETA: 0s - loss: 0.2097 - accuracy: 0.9406
Epoch 2: val_accuracy improved from 0.93300 to 0.94783, saving model to best_model1.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.2085 - accuracy: 0.9408 - val_loss: 0.1821 - val_accuracy: 0.9478
Epoch 3/100
369/375 [============================>.] - ETA: 0s - loss: 0.1554 - accuracy: 0.9557
Epoch 3: val_accuracy improved from 0.94783 to 0.95767, saving model to best_model1.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.1550 - accuracy: 0.9557 - val_loss: 0.1455 - val_accuracy: 0.9577
Epoch 4/100
369/375 [============================>.] - ETA: 0s - loss: 0.1225 - accuracy: 0.9644
Epoch 4: val_accuracy improved from 0.95767 to 0.96467, saving model to best_model1.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.1222 - accuracy: 0.9645 - val_loss: 0.1233 - val_accuracy: 0.9647
Epoch 5/100
363/375 [============================>.] - ETA: 0s - lo

In [4]:
model = keras.models.load_model('best_model3.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.06812819093465805
Test accuracy: 0.979200005531311


## Learning Rates Schedulers

Los learning rate schedulers son mecanismos de modificación del learning rate. Normalmente lo que hacen es reducir el valor del learning rate, lo que se conoce como ***learning rate annealing***. Esta modificación se suele realizar al acabar cada epoch.

Keras ya dispone de algunos learning rate schedulers implementados pero el usuario puede implemetar su propia estrategia de annealing. Veamos ambos casos.

### LRS ya implementado en Keras

In [6]:
# Emplear un LRS estandard de Keras: ReduceLROnPlateau

from keras.callbacks import ReduceLROnPlateau #importar importante

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath="best_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)

#monitor, te fijas es la perdida de validacio, cuando la validacion varia menos de min_delta(definida por default) durante 2 epochs,
#reduces por el LR multiplicando factor,en este caso a 0.5, el minlr sirve para saber hasta cuando reducir el LR

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                     callbacks=[reduce_lr,checkpoint])  ## <--- aquí está



Epoch 1/100
370/375 [============================>.] - ETA: 0s - loss: 0.3412 - accuracy: 0.9135
Epoch 1: val_accuracy improved from -inf to 0.94708, saving model to best_model.h5
375/375 [==============================] - 3s 6ms/step - loss: 0.3391 - accuracy: 0.9139 - val_loss: 0.1817 - val_accuracy: 0.9471 - lr: 0.0100
Epoch 2/100
 25/375 [=>............................] - ETA: 1s - loss: 0.1357 - accuracy: 0.9625

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


371/375 [============================>.] - ETA: 0s - loss: 0.1523 - accuracy: 0.9559
Epoch 2: val_accuracy improved from 0.94708 to 0.95175, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.1528 - accuracy: 0.9558 - val_loss: 0.1838 - val_accuracy: 0.9517 - lr: 0.0100
Epoch 3/100
371/375 [============================>.] - ETA: 0s - loss: 0.1257 - accuracy: 0.9645
Epoch 3: val_accuracy improved from 0.95175 to 0.95200, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.1261 - accuracy: 0.9643 - val_loss: 0.1823 - val_accuracy: 0.9520 - lr: 0.0100
Epoch 4/100
365/375 [============================>.] - ETA: 0s - loss: 0.0598 - accuracy: 0.9818
Epoch 4: val_accuracy improved from 0.95200 to 0.97033, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.0599 - accuracy: 0.9819 - val_loss: 0.1055 - val_accuracy: 0.9703 - lr: 0.0020
Epoch 5/100
368/375 [==========

### LRS propio mediante función

La función toma como entrada el epoch

Cuidado con utilizar LRS y early stopping ya que puede q salga por early sin embargo aun podia mejorar el LR, solo utilizar una de las 2.


In [7]:
## Emplear un LRS propio: LearningRateScheduler

from keras.callbacks import LearningRateScheduler
#muy facil
def scheduler(epoch):
    if epoch < 5:
        return lr
        #a partir de la 5 devuelve el LR *0.1 como si fuera una funcion escalon
    else:
        return lr * 0.1
#para las epoch se usa un poco a ojo y ver cuando crees que has llegado a la meseta
LRS=LearningRateScheduler(scheduler, verbose=1)

model = Sequential()

model.add(Input(784))

model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])






Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/100
375/375 [==============================] - 3s 5ms/step - loss: 0.3342 - accuracy: 0.9139 - val_loss: 0.2074 - val_accuracy: 0.9351 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/100
375/375 [==============================] - 2s 4ms/step - loss: 0.1488 - accuracy: 0.9579 - val_loss: 0.1913 - val_accuracy: 0.9490 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.01.
Epoch 3/100
375/375 [==============================] - 2s 6ms/step - loss: 0.1357 - accuracy: 0.9626 - val_loss: 0.1457 - val_accuracy: 0.9597 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.01.
Epoch 4/100
375/375 [==============================] - 2s 6ms/step - loss: 0.1162 - accuracy: 0.9671 - val_loss: 0.1430 - val_accuracy: 0.9649 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.01.
Epoch 5/100
375/375 [==============================] - 2s 5ms

KeyboardInterrupt: ignored

### LRS propio tipo CosineAnnealing

![cosine annealing](https://github.com/RParedesPalacios/APR_LAB/blob/main/P3/cosine.png?raw=1)


In [8]:
# Emplear un LRS propio, CosineAnnealingScheduler
#BASTANTE BUENA::::::::::::::::::::::::::::::::::::::::::::
from keras.callbacks import LearningRateScheduler
import math

lr_max=0.001
lr_min=0.00001
epochs=100
def cosine_annealing(x): # recordemos, x es el número de epoch
    lr = lr_max/2 * (1 + math.cos(math.pi * x / epochs))#normalizar para que independientemente de la epoch la forma se mantenga del LR
    if lr<lr_min:
        lr=lr_min
    return lr

LRS = LearningRateScheduler(cosine_annealing)

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])



Epoch 1/100
375/375 [==============================] - 3s 5ms/step - loss: 0.2094 - accuracy: 0.9381 - val_loss: 0.1209 - val_accuracy: 0.9630 - lr: 0.0010
Epoch 2/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0817 - accuracy: 0.9743 - val_loss: 0.0779 - val_accuracy: 0.9766 - lr: 9.9975e-04
Epoch 3/100
375/375 [==============================] - 2s 6ms/step - loss: 0.0497 - accuracy: 0.9843 - val_loss: 0.1092 - val_accuracy: 0.9678 - lr: 9.9901e-04
Epoch 4/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0334 - accuracy: 0.9887 - val_loss: 0.0810 - val_accuracy: 0.9783 - lr: 9.9778e-04
Epoch 5/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0296 - accuracy: 0.9901 - val_loss: 0.0770 - val_accuracy: 0.9814 - lr: 9.9606e-04
Epoch 6/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0203 - accuracy: 0.9937 - val_loss: 0.0936 - val_accuracy: 0.9782 - lr: 9.9384e-04
Epoch 7/100
375/375 [=======================

### LRS propio tipo CosineAnnealing with restarts

![cosine annealing restarts](https://github.com/RParedesPalacios/APR_LAB/blob/main/P3/cosinerestart.png?raw=1)

Este scheduler tiene sentido cuanto queremos guardarnos cada uno de los estado del modelo alcanzado en el mínimo LR para luego combinarlos.



Con esta tecnica no encuentras solo un minimo sino que la idea es visitar varios minimos locales

In [9]:
# Emplear un LRS propio, CosineAnnealingScheduler with restarts

from keras.callbacks import LearningRateScheduler
import math

lr_max=0.01
lr_min=0.0001
epochs=250

def cosine_annealing_with_restarts(x):
    lr = lr_max/2 * (1 + math.cos(math.pi * (x % (epochs/5)) / (epochs/5)))
    if lr<lr_min:
        lr=lr_min
    return lr

LRS = LearningRateScheduler(cosine_annealing_with_restarts)

model = Sequential()
model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])

Epoch 1/100
375/375 [==============================] - 3s 5ms/step - loss: 0.2135 - accuracy: 0.9346 - val_loss: 0.1126 - val_accuracy: 0.9671 - lr: 0.0010
Epoch 2/100
375/375 [==============================] - 2s 4ms/step - loss: 0.0821 - accuracy: 0.9744 - val_loss: 0.0787 - val_accuracy: 0.9768 - lr: 9.9384e-04
Epoch 3/100
375/375 [==============================] - 2s 7ms/step - loss: 0.0489 - accuracy: 0.9842 - val_loss: 0.0762 - val_accuracy: 0.9775 - lr: 9.7553e-04
Epoch 4/100
375/375 [==============================] - 2s 4ms/step - loss: 0.0331 - accuracy: 0.9891 - val_loss: 0.0820 - val_accuracy: 0.9791 - lr: 9.4550e-04
Epoch 5/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0237 - accuracy: 0.9924 - val_loss: 0.0768 - val_accuracy: 0.9793 - lr: 9.0451e-04
Epoch 6/100
375/375 [==============================] - 2s 4ms/step - loss: 0.0166 - accuracy: 0.9948 - val_loss: 0.0959 - val_accuracy: 0.9781 - lr: 8.5355e-04
Epoch 7/100
375/375 [=======================

## Ejercicio

Prueba los diferentes LRS sobre el modelo anterior. Emplea la que creas que es la mejor topología de red y optimizador. Emplea model checkpoint. Calcula el accuracy sobre el test con el modelo almacenado en el model checkpoint.



In [10]:
## Solución
#Lo mejor 3 capas de ocultas de 1024 con adam(lr=0.01) LRS-> CosineWithRestarts  5 restarts epochs = 250

# Emplear un LRS propio, CosineAnnealingScheduler with restarts

from keras.callbacks import LearningRateScheduler
import math

lr_max=0.01
lr_min=0.00001
epochs=250

def cosine_annealing_with_restarts(x):
    lr = lr_max/2 * (1 + math.cos(math.pi * (x % (epochs/5)) / (epochs/5)))
    if lr<lr_min:
        lr=lr_min
    return lr

LRS = LearningRateScheduler(cosine_annealing_with_restarts)
checkpoint = keras.callbacks.ModelCheckpoint(filepath="best_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
model = Sequential()
model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS,checkpoint])

#Con el mejor modelo ejecutar el test
model = keras.models.load_model('best_model3.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/250
375/375 [==============================] - ETA: 0s - loss: 0.4602 - accuracy: 0.8944
Epoch 1: val_accuracy improved from -inf to 0.94100, saving model to best_model.h5
375/375 [==============================] - 4s 7ms/step - loss: 0.4602 - accuracy: 0.8944 - val_loss: 0.2098 - val_accuracy: 0.9410 - lr: 0.0100
Epoch 2/250
371/375 [============================>.] - ETA: 0s - loss: 0.1947 - accuracy: 0.9462
Epoch 2: val_accuracy improved from 0.94100 to 0.94925, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.1950 - accuracy: 0.9462 - val_loss: 0.1960 - val_accuracy: 0.9492 - lr: 0.0100
Epoch 3/250
370/375 [============================>.] - ETA: 0s - loss: 0.1616 - accuracy: 0.9552
Epoch 3: val_accuracy improved from 0.94925 to 0.95083, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.1615 - accuracy: 0.9552 - val_loss: 0.1810 - val_accuracy: 0.9508 - lr: 0.0100
Epoch 4/250
366/375 [=

---

## EXTRA, estandaricemos la clase MLP

Vamos a implementar una clase para ahorrarnos escribir código recurrentemente.
Para ello empleamos la **functional api** de Keras:

In [ ]:
from tensorflow import keras
from keras.layers import Dense

class MLP(keras.Model):#hereda modelo de keras

  def __init__(self, input_size,num_classes,hidden=[128]):
    super().__init__()

    self.hidden = []
    self.num_h=len(hidden)

    for h in hidden:
       self.hidden.append(Dense(h, activation='relu'))
    self.out = Dense(num_classes, activation='softmax')

#como se porcesa la informacio a traves de esta red como se comportan las capas
  def call(self, inputs, training=False):
    # a la primera capa oculta le pasas inputs q genera una salida x
    x=self.hidden[0](inputs)
    # a partir de la segunda capa oculta generas las x de cada una de las capas
    for h in range(1,self.num_h):
        x = self.hidden[h](x)
   #con esto puedes tener topologias que no sean lineales . Pej sumar la primera y la ultima si tienen la misma dimension
    x = self.out(x)
    return x



model = MLP(784,10,[1024,512])  ## <-- aquí se instancia el modelo, input, num clases y lista con hidden layers
model.build((None,784)) ## Esto es necesario para poder instaciar adecuadamente todos los shapes del grafo de computación
#none es el tamaño del batch pero nos da igual el valor de este para la creacion de la red neuronal.

model.summary()